In [ ]:
# The Snowpark package is required for Python Worksheets.
# You can add more packages by selecting them using the Packages control and then importing them.

import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import json_extract_path_text,call_udf,md5,parse_json,when_not_matched,lit

def main(session: snowpark.Session):
    try:
        extract_new_sales = (
        session.table("new_sales_transient")
        .withColumn("credit_card_number", json_extract_path_text("$1", lit("creditCardNumber")))
        .withColumn("credit_card_type", json_extract_path_text("$1", lit("creditCardType")))
        .withColumn("date_time", json_extract_path_text("$1", lit("dateTime")))
        .withColumn("id", md5(json_extract_path_text("$1", lit("id"))))
        .withColumn("sale_amount", json_extract_path_text("$1", lit("saleAmount")))
        .select("credit_card_number", "credit_card_type", "date_time", "id", "sale_amount")
        )

        extract_new_sales_items = (
        session.table("new_sales_items_transient")
        .withColumn("credit_card_number", json_extract_path_text("$1", lit("creditCardNumber")))
        .withColumn("date_time", json_extract_path_text("$1", lit("dateTime")))
        .withColumn("id", md5(json_extract_path_text("$1", lit("id"))))
        .withColumn("items", call_udf("sort_alphabetically",(parse_json(json_extract_path_text("$1", lit("items"))))))
        .select("credit_card_number", "date_time", "id", "items")
        )

        join_new_sales_and_sales_items = (
            extract_new_sales.join(
                right= extract_new_sales_items,
                on= extract_new_sales.id == extract_new_sales_items.id,
                rsuffix = "_si"
            )
            .select(
                "credit_card_number",
                "date_time", "id", "items",
                "sale_amount"
            )
        )


        target=session.table("sales_data")

        merged =target.merge(
            source=join_new_sales_and_sales_items,
            join_expr = (target["ID"]==join_new_sales_and_sales_items["ID"]),
            clauses=[
                when_not_matched().insert(
                    {
                        "CREDIT_CARD_NUMBER":join_new_sales_and_sales_items["CREDIT_CARD_NUMBER"],
                        "DATE_TIME":join_new_sales_and_sales_items["DATE_TIME"],
                        "ID":join_new_sales_and_sales_items["ID"],
                        "ITEMS":join_new_sales_and_sales_items["ITEMS"],
                        "SALE_AMOUNT":join_new_sales_and_sales_items["SALE_AMOUNT"]
                    }
                )
                     ]
        )
        return merged
    except Exception as e:
            return e